<a href="https://colab.research.google.com/github/merlinymy/mealPlanGenerator/blob/main/DataCleaningDone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd, json

In [2]:
df = pd.read_json('foundationDownload.json')

In [3]:
with open('foundationDownload.json', 'r') as f:
  data = json.load(f)

In [4]:
foundation_foods = data.get('FoundationFoods', [])

In [5]:
first_foundation_food = foundation_foods[0]
first_foundation_food.keys()

dict_keys(['foodClass', 'description', 'foodNutrients', 'foodAttributes', 'nutrientConversionFactors', 'isHistoricalReference', 'ndbNumber', 'dataType', 'foodCategory', 'fdcId', 'foodPortions', 'publicationDate', 'inputFoods'])

In [ ]:
## key values to keep
# description field is a type of <class 'str'>
# foodNutrients field is a type of <class 'list'>
# ndbNumber field is a type of <class 'int'>
# foodCategory field is a type of <class 'dict'>
# fdcId field is a type of <class 'int'>
# inputFoods field is a type of <class 'list'>

<class 'dict_keys'>


In [23]:
extracted_data = []
for item in foundation_foods:
  extracted_item = {
      'description': item.get('description', ''),
      'foodNutrients': item.get('foodNutrients', []),
      'ndbNumber': item.get('ndbNumber', None),
      'foodCategory': item.get('foodCategory', {}),
      'fdcId': item.get('fdcId', None),
      'inputFoods': item.get('inputFoods', [])
  }
  extracted_data.append(extracted_item)

# Creating a DataFrame from the extracted data
extracted_df = pd.DataFrame(extracted_data)


In [26]:
# Extracting 'description' from 'foodCategory' and adding it as a new column
extracted_df['foodCategory'] = extracted_df['foodCategory'].apply(lambda x: x.get('description', ''))

# Creating separate DataFrames for 'foodNutrients' and 'inputFoods'
food_nutrients_data = []
input_foods_data = []

for index, row in extracted_df.iterrows():
    fdc_id = row['fdcId']

    # Extracting 'foodNutrients' details
    for nutrient in row['foodNutrients']:
        nutrient['fdcId'] = fdc_id
        food_nutrients_data.append(nutrient)

    # Extracting 'inputFoods' details
    for input_food in row['inputFoods']:
        input_food['fdcId'] = fdc_id
        input_foods_data.append(input_food)

# Creating DataFrames
food_nutrients_df = pd.DataFrame(food_nutrients_data)
input_foods_df = pd.DataFrame(input_foods_data)




In [17]:
# Simplifying the food_nutrients_df by removing unnecessary columns and extracting only 'name' and 'unitName' from 'nutrient'

# Dropping unwanted columns
simplified_nutrients_df = food_nutrients_df.drop(['dataPoints', 'foodNutrientDerivation', 'max', 'min', 'footnote', 'median'], axis=1)


In [18]:
# Extracting 'name' and 'unitName' from 'nutrient' column
simplified_nutrients_df['nutrient_name'] = simplified_nutrients_df['nutrient'].apply(lambda x: x.get('name', ''))
simplified_nutrients_df['nutrient_unit'] = simplified_nutrients_df['nutrient'].apply(lambda x: x.get('unitName', ''))

# Dropping the original 'nutrient' column
simplified_nutrients_df = simplified_nutrients_df.drop(['nutrient'], axis=1)

simplified_nutrients_df.head()  # Displaying the first few rows of the simplified DataFrame


,type,id,amount,fdcId,nutrient_name,nutrient_unit
0,FoodNutrient,2219707,3.0,321358,"Cryptoxanthin, beta",µg
1,FoodNutrient,2219708,0.0,321358,Lycopene,µg
2,FoodNutrient,2219709,1.3,321358,"Tocopherol, delta",mg
3,FoodNutrient,2219710,0.0,321358,"Tocotrienol, gamma",mg
4,FoodNutrient,2219711,0.0,321358,"Tocotrienol, delta",mg


In [30]:
main_df = extracted_df.drop(['foodNutrients','ndbNumber','inputFoods'], axis=1)

In [31]:
main_df

,description,foodCategory,fdcId
0,"Hummus, commercial",Legumes and Legume Products,321358
1,"Tomatoes, grape, raw",Vegetables and Vegetable Products,321360
2,"Beans, snap, green, canned, regular pack, drai...",Vegetables and Vegetable Products,321611
3,"Frankfurter, beef, unheated",Sausages and Luncheon Meats,323121
4,"Nuts, almonds, dry roasted, with salt added",Nut and Seed Products,323294
...,...,...,...
260,"Cheese, pasteurized process cheese food or pro...",Dairy and Egg Products,2647439
261,"Cheese, provolone, sliced",Dairy and Egg Products,2647440
262,"Cheese, oaxaca, solid",Dairy and Egg Products,2647441
263,"Cheese, queso fresco, solid",Dairy and Egg Products,2647442


In [32]:
simplified_nutrients_df

,type,id,amount,fdcId,nutrient_name,nutrient_unit
0,FoodNutrient,2219707,3.0,321358,"Cryptoxanthin, beta",µg
1,FoodNutrient,2219708,0.0,321358,Lycopene,µg
2,FoodNutrient,2219709,1.3,321358,"Tocopherol, delta",mg
3,FoodNutrient,2219710,0.0,321358,"Tocotrienol, gamma",mg
4,FoodNutrient,2219711,0.0,321358,"Tocotrienol, delta",mg
...,...,...,...,...,...,...
12908,FoodNutrient,33296880,NaN,2647443,Vitamin A,mg
12909,FoodNutrient,33296881,0.0,2647443,Vitamin D (D2 + D3),µg
12910,FoodNutrient,33296882,351.0,2647443,Energy (Atwater General Factors),kcal
12911,FoodNutrient,33296883,352.0,2647443,Energy (Atwater Specific Factors),kcal


In [33]:
main_df = pd.DataFrame(main_df)
nutrients_df = pd.DataFrame(simplified_nutrients_df)

In [41]:
# Extracting all unique nutrient names from the original simplified_nutrients_df
unique_nutrient_names = nutrients_df['nutrient_name'].unique()
unique_nutrient_names.sort()  # Sorting the nutrient names for better readability
unique_nutrient_names


array([' Ergosta-5,7-dienol', ' Ergosta-7,22-dienol',
       '10-Formyl folic acid (10HCOFA)', '25-hydroxycholecalciferol',
       '5-Formyltetrahydrofolic acid (5-HCOH4',
       '5-methyl tetrahydrofolate (5-MTHF)', 'Alanine', 'Arginine', 'Ash',
       'Aspartic acid', 'Beta-glucan', 'Beta-sitostanol',
       'Beta-sitosterol', 'Betaine', 'Biotin', 'Boron, B',
       'Brassicasterol', 'Calcium, Ca', 'Campestanol', 'Campesterol',
       'Carbohydrate, by difference', 'Carbohydrate, by summation',
       'Carotene, alpha', 'Carotene, beta', 'Carotene, gamma',
       'Cholesterol', 'Choline, free',
       'Choline, from glycerophosphocholine',
       'Choline, from phosphocholine',
       'Choline, from phosphotidyl choline',
       'Choline, from sphingomyelin', 'Choline, total', 'Citric acid',
       'Cobalt, Co', 'Copper, Cu', 'Cryptoxanthin, alpha',
       'Cryptoxanthin, beta', 'Cysteine', 'Cystine', 'Daidzein',
       'Daidzin', 'Delta-5-avenasterol', 'Delta-7-Stigmastenol', 'Energ

In [43]:
# Creating a new DataFrame for nutrients focusing on common and important nutrients
# Including macronutrients, vitamins, minerals, and other relevant compounds

# List of important nutrients to include
important_nutrients_list = [
    'Protein', 'Total fat \(NLEA\)', 'Total lipid \(fat\)', 'Carbohydrate', 'Fiber', 'Sugars',
    'Calcium', 'Iron', 'Potassium', 'Magnesium', 'Zinc',
    'Vitamin A', 'Vitamin C', 'Vitamin D', 'Vitamin E', 'Vitamin K',
    'Thiamin', 'Riboflavin', 'Niacin', 'Vitamin B-6', 'Folate', 'Vitamin B-12',
    'Cholesterol', 'Sodium', 'Lactose', 'Gluten',
    'Saturated Fat', 'Monounsaturated Fat', 'Polyunsaturated Fat', 'Trans Fat'
]

# Creating the filtered DataFrame
important_nutrients_df = nutrients_df[
    nutrients_df['nutrient_name'].str.contains('|'.join(important_nutrients_list), regex=True)
]

important_nutrients_df



,type,id,amount,fdcId,nutrient_name,nutrient_unit
5,FoodNutrient,2219712,0.000,321358,"Vitamin C, total ascorbic acid",mg
6,FoodNutrient,2219713,0.150,321358,Thiamin,mg
7,FoodNutrient,2219714,0.115,321358,Riboflavin,mg
8,FoodNutrient,2219715,36.000,321358,"Folate, total",µg
9,FoodNutrient,2219716,0.000,321358,Vitamin K (Dihydrophylloquinone),µg
...,...,...,...,...,...,...
12906,FoodNutrient,33296878,23.800,2647443,Protein,g
12907,FoodNutrient,33296879,2.720,2647443,"Carbohydrate, by difference",g
12908,FoodNutrient,33296880,NaN,2647443,Vitamin A,mg
12909,FoodNutrient,33296881,0.000,2647443,Vitamin D (D2 + D3),µg


In [44]:
# Checking if all 265 food items from extracted_df are represented in the filtered_nutrients_df
# We do this by comparing the unique fdcIds in both DataFrames

# Unique fdcIds in extracted_df
unique_fdcIds_extracted = set(extracted_df['fdcId'])

# Unique fdcIds in filtered_nutrients_df
unique_fdcIds_filtered_nutrients = set(important_nutrients_df['fdcId'])

# Checking if all fdcIds in extracted_df are in filtered_nutrients_df
missing_fdcIds = unique_fdcIds_extracted - unique_fdcIds_filtered_nutrients
missing_fdcIds


set()

#EDA

In [34]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   description   265 non-null    object
 1   foodCategory  265 non-null    object
 2   fdcId         265 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 6.3+ KB


In [45]:
important_nutrients_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4108 entries, 5 to 12912
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   type           4108 non-null   object 
 1   id             4108 non-null   int64  
 2   amount         4088 non-null   float64
 3   fdcId          4108 non-null   int64  
 4   nutrient_name  4108 non-null   object 
 5   nutrient_unit  4108 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 224.7+ KB


In [46]:
# Checking for missing values in both DataFrames
missing_values_extracted = extracted_df.isnull().sum()
missing_values_nutrients = important_nutrients_df.isnull().sum()

# Descriptive statistics for numerical columns
stats_extracted = extracted_df.describe()
stats_nutrients = important_nutrients_df.describe()

# Unique values analysis for categorical columns
unique_categories_extracted = extracted_df['foodCategory'].nunique()
unique_nutrients = important_nutrients_df['nutrient_name'].nunique()

missing_values_extracted, missing_values_nutrients, stats_extracted, stats_nutrients, unique_categories_extracted, unique_nutrients


(description      0
 foodNutrients    0
 ndbNumber        0
 foodCategory     0
 fdcId            0
 inputFoods       0
 dtype: int64,
 type              0
 id                0
 amount           20
 fdcId             0
 nutrient_name     0
 nutrient_unit     0
 dtype: int64,
            ndbNumber         fdcId
 count     265.000000  2.650000e+02
 mean    37092.830189  1.546317e+06
 std     41023.251772  8.829703e+05
 min      1001.000000  3.213580e+05
 25%      9206.000000  7.467800e+05
 50%     14091.000000  1.999628e+06
 75%    100257.000000  2.346403e+06
 max    100323.000000  2.647443e+06,
                  id        amount         fdcId
 count  4.108000e+03   4088.000000  4.108000e+03
 mean   1.750127e+07     64.094961  1.454372e+06
 std    1.169499e+07    631.981683  8.857545e+05
 min    2.219712e+06     -0.594000  3.213580e+05
 25%    8.514498e+06      0.234000  7.467650e+05
 50%    2.111543e+07      3.240000  1.750341e+06
 75%    2.891219e+07     22.100000  2.346389e+06
 max   

In [47]:
# Filtering the entries in simplified_nutrients_df with negative values for 'amount'
negative_amount_entries = important_nutrients_df[important_nutrients_df['amount'] < 0]
negative_amount_entries


,type,id,amount,fdcId,nutrient_name,nutrient_unit
12539,FoodNutrient,33295318,-0.482,2646168,"Carbohydrate, by difference",g
12562,FoodNutrient,33295322,-0.594,2646169,"Carbohydrate, by difference",g
12583,FoodNutrient,33295326,-0.368,2646170,"Carbohydrate, by difference",g
12606,FoodNutrient,33295330,-0.406,2646171,"Carbohydrate, by difference",g
12628,FoodNutrient,33295334,-0.306,2646172,"Carbohydrate, by difference",g
12672,FoodNutrient,33295342,-0.524,2646174,"Carbohydrate, by difference",g
12694,FoodNutrient,33295346,-0.304,2646175,"Carbohydrate, by difference",g


In [48]:
# export the data
nutrients_file_path = 'nutrients_cleaned.csv'
food_main_file_path = 'food_main.csv'
important_nutrients_df.to_csv(nutrients_file_path, index=False)
main_df.to_csv(food_main_file_path, index=False)